In [1]:
from datasets import Dataset
from ReadLoad import read_json
from prompt_template import get_input_template, get_sys_prompt
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
data = read_json('dataset/train.json')
train_data = [{
    'Instraction': '对用户提出的有关保险条款的问题给予准确、清晰的回答。',
    'Input': get_input_template(d['产品名'], d['条款'], d['问题']),
    'Ouput': d['答案']
} for d in data]
df = pd.DataFrame(train_data)
ds = Dataset.from_pandas(df)

{'Instraction': '你是一个基于保险条款的问答系统，对用户提出的有关保险条款的问题给予准确、清晰的回答。',
 'Input': '产品名称："""优医保·高端门诊住院医疗"""\n保险条款："""【中国境内】 指除香港特别行政区、澳门特别行政区以及台湾地区之外的中华人民共和国领土。"""\n用户问题："""【中国境内】在优医保·高端门诊住院医疗保险产品中指的是哪些地区？"""',
 'Ouput': '在优医保·高端门诊住院医疗保险产品中，【中国境内】指的是除香港特别行政区、澳门特别行政区以及台湾地区之外的中华人民共和国领土。'}

In [8]:
tokenizer = AutoTokenizer.from_pretrained('/root/autodl-tmp/qwen/Qwen2-7B-Instruct', use_fast=False, trust_remote_code=True)
tokenizer

{'Instraction': ['你是一个基于保险条款的问答系统，对用户提出的有关保险条款的问题给予准确、清晰的回答。',
  '你是一个基于保险条款的问答系统，对用户提出的有关保险条款的问题给予准确、清晰的回答。',
  '你是一个基于保险条款的问答系统，对用户提出的有关保险条款的问题给予准确、清晰的回答。'],
 'Input': ['产品名称："""优医保·高端门诊住院医疗"""\n保险条款："""【中国境内】 指除香港特别行政区、澳门特别行政区以及台湾地区之外的中华人民共和国领土。"""\n用户问题："""【中国境内】在优医保·高端门诊住院医疗保险产品中指的是哪些地区？"""',
  '产品名称："""同佑e生(臻享版)终身重疾险"""\n保险条款："""中度再生障碍性贫血 指因骨髓造血功能慢性持续性衰竭导致的贫血、中性粒细胞减少及血小板减少。须满足下列至少一项条件,但未达到本合同所指重大疾病“重型再生障碍性贫血”的给付标准：1.骨髓刺激疗法至少一个月；2.免疫抑制剂治疗至少一个月。"""\n用户问题："""中度再生障碍性贫血会导致哪些症状？"""',
  '产品名称："""众民保·百万医疗（免健告）"""\n保险条款："""认可的互联网药店 经保险人审核认可，取得互联网药品信息服务资格证书，为被保险人提供购药或者配送服务的药店，具体名单在保险人官方网站或销售平台公示。"""\n用户问题："""互联网药店需要取得哪种证书才能被保险公司认可？"""'],
 'Ouput': ['在优医保·高端门诊住院医疗保险产品中，【中国境内】指的是除香港特别行政区、澳门特别行政区以及台湾地区之外的中华人民共和国领土。',
  '中度再生障碍性贫血会导致贫血、中性粒细胞减少及血小板减少。',
  '互联网药店需要取得互联网药品信息服务资格证书才能被保险公司认可。']}

In [ ]:
def process_func(example):
    MAX_LENGTH = 384    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(f"<|im_start|>system\n你是一个基于保险条款的问答系统<|im_end|>\n<|im_start|>user\n{example['instruction'] + example['input']}<|im_end|>\n<|im_start|>assistant\n", add_special_tokens=False)  # add_special_tokens 不在开头加 special_tokens
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # 因为eos token咱们也是要关注的所以 补充为1
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]  
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

In [ ]:
import torch

model = AutoModelForCausalLM.from_pretrained('/root/autodl-tmp/qwen/Qwen2-7B-Instruct', device_map="auto",torch_dtype=torch.bfloat16)
model.enable_input_require_grads() # 开启梯度检查点时，要执行该方法

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    #target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
model = get_peft_model(model, config)
model.print_trainable_parameters()

In [ ]:
args = TrainingArguments(
    output_dir="./output/Qwen2_instruct_lora",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=10,
    num_train_epochs=3,
    save_steps=100, # 为了快速演示，这里设置10，建议你设置成100
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True
)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)
trainer.train()